# Pipeline de demonstração do EA‑VRP.

Gera dados sintéticos (passageiros, veículos, pontos de recarga), executa o
agrupamento geográfico e imprime no console a composição de cada grupo.

Uso
---
```
$ docker compose up 
```

In [1]:
import random
from typing import List

from geografics import generate_random_geografic_points
from grouping import GeoGrouper, EAVRPGroup
from utils import Passenger, Vehicle, RechargePoint
from encoder import EA_VRP_QUBOEncoder
from solver import QAOASolver
from graph import build_graph

from pathlib import Path

## Helpers de geração sintética

In [2]:
def make_passengers(n: int, x_lim: List, y_lim: List) -> List[Passenger]:
    """Cria *n* passageiros com origem e destino aleatórios."""
    origin = generate_random_geografic_points(n, x_lim, y_lim)
    dest = generate_random_geografic_points(n, x_lim, y_lim)
    passengers: List[Passenger] = []
    for i, (o, d) in enumerate(zip(origin,dest)):
        passengers.append(Passenger(id=i, origin=o, destination=d))
    return passengers

In [3]:
def make_vehicles(n: int) -> List[Vehicle]:
    """Cria *n* veículos com parâmetros padrão."""
    return [Vehicle(id=i + 1) for i in range(n)]

In [4]:
def make_recharge_points(n: int, x_lim: List, y_lim: List) -> List[RechargePoint]:
    origin = generate_random_geografic_points(n, x_lim, y_lim)
    recharge_points: List[RechargePoint] = []
    for i, o in enumerate(origin):
        recharge_points.append(RechargePoint(id=i, location=o))
    return recharge_points

In [5]:
OUTDIR = Path("figs")
OUTDIR.mkdir(parents=True, exist_ok=True)

n_passengers = 10
n_vehicles = 8
n_recharge_points = 5

In [6]:
p=2
shots=1000
steps=15
diameter = 0.2 # 1 => 100kmprint(depot)
shrink_epsilon = 0.9 # None

In [7]:
seed = 42
filename_hist = OUTDIR / "teste_histograma.png" # nome do arquivo do histograma
filename_graph = OUTDIR / "teste_grafo.png" # nome do arquivo do grafo

In [8]:
passengers = make_passengers(n_passengers, (-24, -23.4), (-46.8, -46.3))
vehicles = make_vehicles(n_vehicles)
recharge_points = make_recharge_points(n_recharge_points, (-24, -23.4), (-46.8, -46.3))
depot = generate_random_geografic_points(1, (-24, -23.4), (-46.8, -46.3))
print(depot)

[(np.float64(-23.919478993200133), np.float64(-46.71171647949851))]


## Agrupamento de Passageiros

In [9]:
grouper = GeoGrouper(delta=diameter)
groups = grouper.fit(passengers)

for g in groups:
    p_ids = [passenger.id for passenger in g.passengers]
    print(f"Grupo {g.id:02d} | {len(p_ids)} passageiros -> {p_ids}")

total_qubits = len(groups)  # variável por grupo (modelo futuro)
print(f"Total de grupos (≈ qubits na formulação): {total_qubits}")

Grupo 01 | 2 passageiros -> [0, 8]
Grupo 02 | 2 passageiros -> [1, 4]
Grupo 03 | 1 passageiros -> [2]
Grupo 04 | 2 passageiros -> [3, 7]
Grupo 05 | 1 passageiros -> [5]
Grupo 06 | 1 passageiros -> [6]
Grupo 07 | 1 passageiros -> [9]
Total de grupos (≈ qubits na formulação): 7


In [10]:
# QUBO para somente grupos sem compressão
encoder = EA_VRP_QUBOEncoder(vehicles, 
                             groups, 
                             recharge_points,
                             depot=depot
                            )
encoder_passengers = EA_VRP_QUBOEncoder(vehicles, 
                             passengers, 
                             recharge_points, 
                             depot=depot
                            )

In [12]:
print(f"Quantidade de qubits com grupos: {encoder.num_qubits}")
print(f"Quantidade de qubits sem grupos: {encoder_passengers.num_qubits}")

Quantidade de qubits com grupos: 96
Quantidade de qubits sem grupos: 120


In [ ]:
print("Matriz QUBO (valores arredondados a 2 casas decimais):")
encoder.print_matrix(precision=2)

In [ ]:
print("Matriz QUBO Somente passageiros (valores arredondados a 2 casas decimais):")
encoder_passengers.print_matrix(precision=2)

In [ ]:
print(f"Offset com compressão (constante): {encoder.offset:.2f}")

In [ ]:
# 3) QAOA 
solver = QAOASolver(encoder,
                    p=p,
                    shots=shots,
                    steps=steps,
                    dev="lightning.qubit", # "default.qubit"
                    seed=seed)
best_bits, best_cost = solver.solve()

In [ ]:
# Estado inteiro x (LSB = qubit 0)
x_val = int("".join(map(str, best_bits[::-1])), 2)
print(f"Melhor estado: |{x_val}⟩  ==> custo {best_cost:.2f}\n")

In [ ]:
# 4) Histograma
solver.plot_histogram(filename_hist)
print(f"Histograma salvo em: {filename_hist}")

In [ ]:
build_graph(passengers, groups, recharge_points, filename_graph)
print(f"Grafo salvo em: {filename_graph}")